# URL Params for the Image-Specific Endpoint

If you've used the Bing Search APIs before, you may be familiar with the `responseFilter=Images` URL parameter. This is useful if you **only** have access to the websearch endpoint. However, if you have access to the image endpoint, you can use a much richer set of filters to get the pictures you're looking for. 

## Setup:

In [1]:
import os
import requests
from IPython.display import Image

In [2]:
API_KEY = os.environ.get('BING_KEY', default='ENTER YOUR API KEY HERE IF YOU DIDNT SET BING_KEY')

In [4]:
print(API_KEY)

c0468feee246448ca66e2fe9aa940772


In [5]:
ENDPOINT = ' https://api.cognitive.microsoft.com/bing/v7.0/images/search?'
HEADERS = {
    'Ocp-Apim-Subscription-Key' : API_KEY
}
STARTING_PARAMS = {
    "q" : None,
    "mkt" : "en-US",
    "offset" : "0",
  }

In [6]:
# Common accross all notebooks - 
def call_api(query, session=None, custom_params=None):
    sesh = session
    if not session:
        sesh = requests.Session()
        sesh.params = PARAMS
        sesh.params.update({'q' : query})
        sesh.headers = HEADERS
    if custom_params:
        sesh.params.update(custom_params)
    return sesh.get(ENDPOINT)

def handle_response(resp):
    assert resp.status_code == 200
    return resp.json()

In [7]:
# Common accross image-centric notebooks - 
def extract_img_urls(response_json):
    return [i['contentUrl'] for i in response_json['value']]

def display(img):
    return Image(url=img)

## Exploring params of the Image-Specific search endpoint:

#### Ex 1: Square drawing of a red car that's free to share commercially

First, let's get a copy of our default URL param dictionary, & just do a normal search for images of red cars.

In [8]:
q = 'red car'
PARAMS = STARTING_PARAMS.copy()

Simple enough. The functions above will ask for pictures of `'red car'`s and give us back some URLs. Let's create a function that ties the three helper functions above together:

In [9]:
def get_images(q, session=None, custom_params=None):
    resp = call_api(q, session=session, custom_params=custom_params)
    json = handle_response(resp)
    return extract_img_urls(json)

Now it's easy for us to get images from Bing using a simple search term:

In [10]:
images = get_images(q)

And voila!

In [11]:
display(images[0])

If you `display()` all the images in `images`, you'll notice that they are mostly real photos. Let's say we're looking for **drawings** of red cars.

Since we're using the image-specific API endpoint, we can pass in params like `'imageType'`

In [12]:
new_param = {'imageType' : 'Clipart'}
images = get_images(q, custom_params=new_param)

In [13]:
display(images[0])

Ok cool. But now let's pretend I'm looking for images to use in my website & need one that's a perfect square. 

In [14]:
new_new_param = {'aspect' : 'square'}
images = get_images(q, custom_params=new_new_param)

In [15]:
display(images[0])

In [16]:
display(images[-1])

Oh no! That last one looks super-proprietary. I'd like also to only find images that are ok for me to use commercially.

In [17]:
newer_params = {
    'license' : 'ShareCommercially'
}
images = get_images(q, custom_params=newer_params)

In [18]:
print(images[1])
display(images[1])

https://images.vexels.com/media/users/3/127817/isolated/lists/4fd0c9dcae60a9a63579b9b6853eeb09-glossy-retro-beetle-car.png


Note that, while useful for free-content aggregation, certain license params can reduce the number of relevant results.

For example, we're now looking for square & non-photographic images of `'red car'`s which their owners have manually tagged as "free to use commercially." Let's see what happens when I try to add `{'size' : 'large'}`:

In [19]:
newest_params = {
    'size' : 'large'
}
images = get_images(q, custom_params=newest_params)

In [20]:
display(images[0])

#### Pictures of People

Let's find large faceshots of people smiling.

In [21]:
q = 'happy'
PARAMS = STARTING_PARAMS.copy()

In [22]:
new_params = {
    'size' : 'Large',
    'imageType' : 'Photo',
    'imageContent' : 'Face' 
}

In [23]:
images = get_images(q, custom_params=new_params)

In [24]:
print(images[0])
display(images[0])

http://i.huffpost.com/gen/1600476/images/o-HAPPY-CHILD-facebook.jpg


#### The Empire State Building

Let's find a tall photo of the empire state building at night

In [25]:
q = 'empire state building at midnight'
PARAMS = STARTING_PARAMS.copy()

In [26]:
new_params = {
    'imageType' : 'Photo',
    'aspect' : 'tall'
}

In [27]:
images = get_images(q, custom_params=new_params)

In [28]:
print(images[1])
display(images[1])

https://2i7bfq1hqqte46nlrr2zpr7h-wpengine.netdna-ssl.com/wp-content/uploads/2017/04/DSC_0007.jpg
